In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['want','got','say','amp', 'thi', 'ain']
stopwords.extend(newStopWords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess(text_string):
  text_string = text_string.translate(string.punctuation)
    ## Convert words to lower case and split them
  text_string = text_string.lower().split()
    ## Remove stop words
  text_string = [w for w in text_string if not w in stopwords and len(w) >= 3]
  text_string = " ".join(text_string) 
  space_pattern = '\s+'
  giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
  mention_regex = '@\w+\W+'
  excliamation_regex = '!+'
  RT_regex = 'RT'
  parsed_text = re.sub(space_pattern, ' ', text_string)
  parsed_text = re.sub(excliamation_regex,'! ',parsed_text)
  parsed_text = re.sub(giant_url_regex, '', parsed_text)
  parsed_text = re.sub(mention_regex, '', parsed_text)
  parsed_text = re.sub(RT_regex,'', parsed_text)
  parsed_text = re.sub('(.)\\1{2,}', '\\1', parsed_text)
  parsed_text = re.sub('bitche', 'bitch',parsed_text)
  parsed_text.replace("#", "")
  parsed_text = parsed_text.split()
  stemmer = SnowballStemmer('english')
  stemmed_words = [stemmer.stem(word) for word in parsed_text]
  parsed_text = " ".join(stemmed_words)
  #parsed_text = parsed_text.code("utf-8", errors='ignore')
  parsed_text = re.sub('[^a-zA-Z]',' ', parsed_text)
  return parsed_text

In [ ]:
# change dt for different models 
from sklearn.ensemble import RandomForestClassifier
def sigmoid(x):
  return (1/(1+np.exp(-1*x)))  
dt = RandomForestClassifier(n_estimators = 100)

def pso(X, Y, popu_size, maxItr=5):
  particles = []
  for i in range(popu_size):
      particles.append(np.random.choice([0, 1], size=1500))
  particles = np.array(particles,dtype=float)
  velocities = np.random.rand(popu_size,1500)
  itr = 0
  pbest = np.zeros((popu_size,1500))
  pbest_score = np.ones((popu_size,),dtype=float)*0
  gbest_score = 0
  gbest = np.zeros((1500,))
  while(itr < maxItr):
    # start = time()
    for i in range(popu_size):
      X_togive = np.multiply(X, particles[i])
      dt.fit(X_togive, Y)
      Xt = np.multiply(X_train, particles[i])
      score = dt.score(X_test,Y_test)
      if score > pbest_score[i]:
        pbest_score[i] = score
        pbest[i] = particles[i]
      if score > gbest_score:
        gbest_score = score
        gbest = pbest[i]
    for i in range(popu_size):
      velocities[i] = 0.4*velocities[i] + 2*np.random.rand()*(pbest[i]-particles[i]) +2*np.random.rand()*(gbest-particles[i])
      for kk in range(1500):
        if velocities[i][kk]>4.0:
          velocities[i][kk] =  4.0
        elif velocities[i][kk]<-4.0:
          velocities[i][kk] = -4.0
      velo_sig = sigmoid(velocities[i])
      for kk in range(97):
        particles[i][kk] = (np.random.randn()<velo_sig[kk])*1
    # end = time()
    print("Epochs:",itr+1,"Best Score:",gbest_score)
    itr+=1
  return gbest

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/MitHar24/dataset/main/New%20combined_Dataset.csv')
df.head()

,Label,Tweet
0,1,@BDUTT Shame on your hate news n fake news.
1,1,@AP Pita can suck my ass! What a shit show of ...
2,0,@simonjames67 @Mr_John_Harvey_ @theJeremyVine ...
3,0,@GMB @SeanFletcherTV Oh wow.........what the h...
4,0,@KallipolisState @talkRADIO @JuliaHB1 @spikedo...


In [ ]:
df['processed'] = df['Tweet'].apply(lambda x: preprocess(x))
df.head()

,Label,Tweet,processed
0,1,@BDUTT Shame on your hate news n fake news.,shame hate news fake news
1,1,@AP Pita can suck my ass! What a shit show of ...,pita suck ass shit show organ put anim save
2,0,@simonjames67 @Mr_John_Harvey_ @theJeremyVine ...,mr john harvey waitros i m littl worri bank b...
3,0,@GMB @SeanFletcherTV Oh wow.........what the h...,seanfletchertv wow what hell know wolv relega...
4,0,@KallipolisState @talkRADIO @JuliaHB1 @spikedo...,talkradio spikedonlin noth like outrag now ma...


In [ ]:
corpus = []
for i in range(0, len(df)):
  review = df['processed'][i]
  # review = re.sub('[^a-zA-Z]',' ', df['processed'][i])
  corpus.append(review)
# Bag Of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
#creating matrix of features
X = cv.fit_transform(corpus).toarray()
Y = df.iloc[:,0].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 0)

In [ ]:
# Scaling the X parameters i.e. X_train and X_test
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [ ]:
Y_train.shape

(55319,)

In [ ]:
type(Y_train)

numpy.ndarray

In [ ]:
X_train.shape

(55319, 1500)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
new_x = pso(X_train, Y_train,5,5)

Epochs: 1 Best Score: 0.774397181341065
Epochs: 2 Best Score: 0.774397181341065
Epochs: 3 Best Score: 0.774397181341065
Epochs: 4 Best Score: 0.774397181341065
Epochs: 5 Best Score: 0.7744705839174955


In [ ]:
X_togive = np.multiply(X_train,new_x)
dt.fit(X_togive, Y_train)
Y_pred = dt.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
f1 = f1_score(Y_test, Y_pred)
ps = precision_score(Y_test, Y_pred)
rs = recall_score(Y_test, Y_pred)
asc = accuracy_score(Y_test, Y_pred)

In [ ]:
print(f"F1 Score {f1}")
print(f"Precesion Score {ps}")
print(f"Recall Score {rs}")
print(f"Accuracy Score {asc}")

F1 Score 0.7256399946387885
Precesion Score 0.8132385339475265
Recall Score 0.6550778414132451
Accuracy Score 0.7746173890703564
